# **Power Bi Vs Tableau**
## **Which one should you learn first?**

The focus of this analysis is to determine which BI tool one should learn first based on job market trends. In this analysis, I will scrap job data from the following job boards:
* **Indeed.com**: This will provide data on global market trends
* **My job Mag Kenya**:provides local context on market trends
* **Upwork**: provides context on the freelance landscape

After scrapping, I will analyse the deman of each BI tool, by applying a regex function to understand which is the most mentioned.



In [33]:
#import all necerssary libraries

from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
from selenium_stealth import stealth

import time
import random

In [34]:
#set up the browser using the options object

options=webdriver.ChromeOptions()
options.add_argument('--start-maximized')
options.add_argument('--disable-blink-features=AutomationControlled')
options.add_experimental_option('excludeSwitches',['enable_automation'])
options.add_experimental_option('useAutomationExtension','False')

driver = webdriver.Chrome(
    service=Service(ChromeDriverManager().install()),
    options=options
)

#now setup the webdriver in stealth mode

stealth(driver,
        languages=["en-US", "en"],
        vendor="Google Inc.",
        platform="Win32",
        webgl_vendor="Intel Inc.",
        renderer="Intel Iris OpenGL Engine",
        fix_hairline=True,
        )


url=('https://www.indeed.com/')
driver.get(url)
time.sleep(random.uniform(4,6))


'''
I had to use selenium to fill in the job and location fields like a human, because there is a CATCHA 
requirement, when you use the url of the page you are directed to after filling in the job & location

'''

# Fill in the search form like a human
what = driver.find_element(By.ID, "text-input-what")
where = driver.find_element(By.ID, "text-input-where")

search_items='data analyst OR business intelligence analyst OR BI developer'

for char in search_items:
    what.send_keys(char)
    time.sleep(random.uniform(0.05,0.1))
    

# Click the search button
#driver.find_element(By.type, "submit").click()
search_button=driver.find_element(By.CSS_SELECTOR, "button[type='submit']")
time.sleep(random.uniform(1.5,2.5))
search_button.click()

time.sleep(random.uniform(5,8))
driver.execute_script('window.scrollBy(0,300)')
time.sleep(random.uniform(2,4))

input("Incase CAPTCHA has been triggered , just proceed to resolve manually, then press Enter to continue with scrapping")

''

In [35]:
seen_jobs = set()  #to store the first characters of the JD
job_data = []

for index in range(15): #loops through the job cards on the page, from the page there are like 15
    try:
        job_cards = driver.find_elements(By.CLASS_NAME, "tapItem")
        card = job_cards[index]

        if index >= len(job_cards):
            print(f"Only {len(job_cards)} cards found. Stopping at index {index}.")
            break

        #click on the actual job card
        driver.execute_script("arguments[0].scrollIntoView();", card)
        time.sleep(random.uniform(1, 2))
        card.click()
        time.sleep(random.uniform(2, 4))

        try:
            title = driver.find_element(By.CSS_SELECTOR, "h2.jobsearch-JobInfoHeader-title > span").text.strip()
        except:
            title = "N/A"

        try:
            desc = driver.find_element(By.ID, "jobDescriptionText").text.strip()
        except:
            desc = "N/A"

        job_fingerprint = (title, desc[:100])

        if job_fingerprint in seen_jobs:
            print(f" Skipping duplicate job: {title}")
            continue

        seen_jobs.add(job_fingerprint)

        job_data.append({
            "title": title,
            "description": desc
        })

        print(f"Job {index+1}: {title}")

    except Exception as e:
        print(f"Error on job {index+1}: {e}")
        time.sleep(random.uniform(1.5, 2.5))
        continue

   


Job 1: Junior Business Intelligence Analyst or BI As
- job post
Job 2: Business Analyst - Data Analyst/Power BI developer
- job post
Job 3: Business Intelligence (BI) Analyst
- job post
Job 4: Data Analyst – Business Intelligence & Reporting Specialist (Power BI & Tableau)
- job post
Job 5: Business Intelligence (BI) Analyst
- job post
Job 6: Sr. Business Intelligence (BI) Data Analyst
- job post
Job 7: Business Intelligence Data Analyst
- job post
Job 8: FamilySearch Business Intelligence Data Analyst
- job post
Job 9: Data Analyst/Power BI Developer
- job post
Job 10: BI Developer and Data Analyst 1
- job post
Job 11: Business Intelligence & Data Visualizations Analyst
- job post
Job 12: Data Analyst/Data Visualization Developer (PSS2), Rowan Online
- job post
Job 13: BUSINESS INTELLIGENCE DATA ANALYST
- job post
Job 14: Business Intelligence Data Analyst
- job post
Job 15: Business Intelligence Analyst
- job post


In [39]:
'''#loop through the job cards on the first page after the search


job_cards = driver.find_elements(By.CLASS_NAME, "tapItem")
print(f"{len(job_cards)} job cards found on this page")'''

'#loop through the job cards on the first page after the search\n\n\njob_cards = driver.find_elements(By.CLASS_NAME, "tapItem")\nprint(f"{len(job_cards)} job cards found on this page")'

In [38]:
import csv

# Save to CSV
with open("indeed_jobs.csv", mode="w", newline="", encoding="utf-8") as file:
    writer = csv.DictWriter(file, fieldnames=["title", "description"])
    writer.writeheader()  # write column headers
    writer.writerows(job_data)  # write job entries

print(f"\n Saved {len(job_data)} jobs to 'indeed_jobs.csv'")



 Saved 15 jobs to 'indeed_jobs.csv'
